In [1]:
import random as rd
import numpy as np
import copy


#this just generates a random sequence of bitstream that is whatever length you want

def rand_bitstream(bitlength):
    bitstream = ""
    for i in range(bitlength):
        temp = str(rd.randint(0,1))
        bitstream += temp
    return bitstream



#because I was lazy and didnt want to write this over and over
def con_bin_str(input1):
    step1 = format(input1,'b')
    step2 = str(step1.zfill(96))#hella paranoid
    step3 = step2[:96]#we had cases of output being 97 bits long... can't have that
    return step3

#Slightly lighter weight than con_bin_str
def intToBin(value,size):
    return bin(value)[2:].zfill(size)

def npStr(nump):
    outStr = ""
    for value in nump:
        outStr+=str(value)
    return outStr
def intToNp(val, size):
    strTemp = intToBin(val,size)
    npOut = np.array([9]*size)
    for i in range(size):
        npOut[i] = int(strTemp[i])
    return npOut
def binToNp(binStr):
    size = len(binStr)
    npOut = np.array([9]*size)
    for i in range(size):
        npOut[i] = int(binStr[i])
    return npOut

#I wanted to define this so that you wouldn't get a random starting IDP_K data stream Every time 
# you did a protocal run.
IDP_K_str = '111111111000100100001110010001011010011111100010010000000111100101111101110110101101000011000111101101001011100000111010110011100001101110111001111111110100010010010001100000011101000110001001000101101001110010000010111110000011001100001100111101101011000010111100110101100110101000110101110001011010111110101010111100101001111001100101000111010110111101111100100011100010101011000010111110011111110011100000111010011011011100111000110100100110100110011011001011111101011011010110'


In [2]:

def MMAP(oracle):
    MOD_SIZE = 2**96
    
    #this is our ID, never changing, always the same
    ID_MMAP_str ='101110011011001011100100001101010011110010001001000101001101001001110010110100100001011001110010'
    
    ID_MMAP = int(ID_MMAP_str,2)
    
    IDP_MMAP_str, K_1_MMAP_str, K_2_MMAP_str, K_3_MMAP_str, K_4_MMAP_str = [oracle[i:i+96] for i in range(0,len(oracle),96)]
    
    IDP_MMAP = int(IDP_MMAP_str,2)
    K_1_MMAP = int(K_1_MMAP_str,2)
    K_2_MMAP = int(K_2_MMAP_str,2)
    K_3_MMAP = int(K_3_MMAP_str,2)
    K_4_MMAP = int(K_4_MMAP_str,2)
    
    
    n1_MMAP = int(rand_bitstream(96),2)
    n2_MMAP = int(rand_bitstream(96),2)
    
    A_int_MMAP = (IDP_MMAP ^ K_1_MMAP ^ n1_MMAP)%MOD_SIZE
    B_int_MMAP = ((IDP_MMAP & K_2_MMAP) | n1_MMAP)%MOD_SIZE
    C_int_MMAP = (IDP_MMAP + K_3_MMAP + n2_MMAP)%MOD_SIZE
    D_int_MMAP = ((IDP_MMAP | K_4_MMAP) & n2_MMAP)%MOD_SIZE
    E_int_MMAP = ((IDP_MMAP + ID_MMAP) ^ n1_MMAP)%MOD_SIZE
    
    NextIDP_int_MMAP = ((IDP_MMAP + (n1_MMAP ^ n2_MMAP)) ^ ID_MMAP)%MOD_SIZE
    NextK_1_int_MMAP = (K_1_MMAP ^ n1_MMAP ^ (K_3_MMAP + ID_MMAP))%MOD_SIZE
    NextK_2_int_MMAP = (K_2_MMAP ^ n2_MMAP ^ (K_4_MMAP + ID_MMAP))%MOD_SIZE
    NextK_3_int_MMAP = ((K_3_MMAP ^ n1_MMAP) + (K_1_MMAP ^ ID_MMAP))%MOD_SIZE
    NextK_4_int_MMAP = ((K_4_MMAP ^ n1_MMAP) + (K_2_MMAP ^ ID_MMAP))%MOD_SIZE
    
    A_MMAP = con_bin_str(A_int_MMAP)
    B_MMAP = con_bin_str(B_int_MMAP)
    C_MMAP = con_bin_str(C_int_MMAP)
    D_MMAP = con_bin_str(D_int_MMAP)
    E_MMAP = con_bin_str(E_int_MMAP)
    
    NextIDP_MMAP = con_bin_str(NextIDP_int_MMAP)
    NextK_1_MMAP = con_bin_str(NextK_1_int_MMAP)
    NextK_2_MMAP = con_bin_str(NextK_2_int_MMAP)
    NextK_3_MMAP = con_bin_str(NextK_3_int_MMAP)
    NextK_4_MMAP = con_bin_str(NextK_4_int_MMAP)
    
    NextIDP_K_MMAP = NextIDP_MMAP + NextK_1_MMAP + NextK_2_MMAP + NextK_3_MMAP + NextK_4_MMAP
    
    return A_MMAP,B_MMAP,C_MMAP,D_MMAP,E_MMAP,NextIDP_K_MMAP
    
    

    
AMMAP,BMMAP,CMMAP,DMMAP,EMMAP,NextIDP_KMMAP = MMAP(IDP_K_str)








In [3]:
def Fp_func(input1):
    chunks = [input1[i:i+4] for i in range(0,len(input1),4)]
    final_output_str = ''
    for chunk in chunks:
        final_output_str += str(int(chunk[0])^int(chunk[1])^int(chunk[2])^int(chunk[3]))
    return final_output_str
    
print(Fp_func("101101101000"))

def Fp_func_old(input1):
    
    chunks = [input1[i:i+4] for i in range(0,len(input1),4)]
    
    final_output_str = ''
    
    for j in range(len(chunks)):
        
        k = 0
        
        while k < len(chunks[j]):
                   
            x1_str, x2_str, x3_str, x4_str = [chunks[j][l] for l in range(len(chunks[j]))]
            
            x1 = int(x1_str,2)
            x2 = int(x2_str,2)
            x3 = int(x3_str,2)
            x4 = int(x4_str,2)
            
            output = ((x1 ^ x2) ^ (x3 ^ x4))
            
            output_binary = format(output,'b')
            output_str = str(output_binary)
            
            k = k + 1
            
        final_output_str += output_str
            
    return final_output_str    
   
    
    

101


In [4]:
# #Finds and outputs the ID of an MMAP RFID system, given only intercepted comms between reader and tag
# SIZE = 96
# def MMAP_Cracker(runtime):
#     garbage = "" #Used to collect useless information, and ignore it
    
#     #Finding the first IDP we will use, so that it works from any random valid IDP
#     #We don't care about the previous A-E, since we are assuming that we don't know the IDP used for that interaction
#     garbage,garbage,garbage,garbage,garbage,IDP_next = MMAP(IDP_K_str)
    
#     #Initializing our ID output to all 9s, to represent that we don't know what it is
#     ID = np.array([9]*SIZE)
    
#       #E_XOR IDP pair array
#     EXOR_NEGIDP = []
    
#     #Until we return a complete ID, keep making MMAP exchanges
#     while(1):
#         runtime+=1
#         #Initializing our IDP
#         IDP = IDP_next[0:SIZE]
#         #Initializing A-E, and the next IDP
#         A,B,C,D,E,IDP_next = MMAP(IDP_next)
        
#         #Initializing n1 to all 9s
#         n1 = np.array([9]*SIZE)
        
#         #Finding all knowable values of n1
#         i = 0
#         for i in range(SIZE):
#             if IDP[i] == '0':
#                 if B[i] == '0':
#                     n1[i] = 0
#                 else:
#                     n1[i] = 1
#        # npPrint(n1)
#         #Creating our negative IDP in two's complement
#         negIDP = ""
#         for i in range(SIZE):
#             if IDP[i] == "0":
#                 negIDP += "1"
#             else:
#                 negIDP += "0"
#         negIDP = bin(int(negIDP,2)+1)[2:].zfill(SIZE)
# #         print("IDP:    ",IDP)
# #         print("negIDP: ",negIDP)
# #         print("IDPCHK: ", bin(int(IDP,2)+int(negIDP,2)))
        
#         #Initializing E XOR n1 to 9s
#         E_xor_n1 = np.array([9]*SIZE)
        
#         #Calculating knowable values of E XOR n1
#     #    print(E)
#         for i in range(SIZE):
#             num = int(E[i],2)
#             if(n1[i] != 9):
#                 E_xor_n1[i] = num^n1[i]
#     #    npPrint(E_xor_n1)
        
        
#         EXOR_NEGIDP.append([E_xor_n1, negIDP])
#         for exorpair in EXOR_NEGIDP:
#             E_xor_n1 = exorpair[0]
#             negIDP = exorpair[1]
            
#         #Filling in the blanks of EXOR using ID    
#             carry = 0
#             for i in range(SIZE-1,-1,-1):
#                 if ID[i] == 9: break

#                 # negIDP 111
#                 # EXOR 001
#                 # ID 000

#                 # ID 000
#                 # negIDP 111
#                 # EXOR 111 WRONG
#                 val = ID[i] + int(negIDP[i],2) + carry
#                 if E_xor_n1[i] == 9:
#                     E_xor_n1[i] = (val)%2
#                 carry = E_xor_n1[i] + int(negIDP[i],2) + carry > 1
#             #print("E_xor_n1 predic: ",npPrint(E_xor_n1))
#                 exorpair[0] = E_xor_n1
#                 exorpair[1] = negIDP
#             #Calculating knowable values of the ID
#                 carry = 0
#                 for i in range(SIZE-1,-1,-1):
#                     if E_xor_n1[i] == 9: break
#                     val = E_xor_n1[i] + int(negIDP[i],2) + carry
#                     if ID[i] == 9:
#                         ID[i] = (val)%2
#                     carry = val > 1
#                 print(ID)
#                 #If ID's most significant bit is found, return it
#                 if ID[0] != 9:
#                     print(runtime)
#                     return ID
# MMAP_Cracker(0)

In [5]:
class EMAPoracle2:
    def __init__(self, size):
        temp = rand_bitstream(size*5)
        idp_emap_str, k1_str, k2_str, k3_str, k4_str = [temp[i:i+size] for i in range(0,len(temp),size)]
        
        self.IDP = int(idp_emap_str,2)
        
        self.ID = int('101110011011001011100100001101010011110010001001000101001101001001110010110100100001011001110010',2)
                
        #create randomized n values 
        self.n1 = int(rand_bitstream(size),2)
        self.n2 = int(rand_bitstream(size),2)
        
        #create the integer Ks in here
        self.k1 = int(k1_str,2)
        self.k2 = int(k2_str,2)
        self.k3 = int(k3_str,2)
        self.k4 = int(k4_str,2)
        self.mod = 2**size
        self.size = size
        
    def helloInternal(self):
        ID_str = intToBin(self.ID, self.size)
#         id_str1, id_str2 = [IDP_str[i:i+self.size//2] for i in range(0,self.size,self.size//2)]
        id_str1 = ID_str[:self.size//2]
        id_str2 = ID_str[self.size//2:]
        
        k1fp = Fp_func(intToBin(self.k1, self.size))
        k2fp = Fp_func(intToBin(self.k2, self.size))
        k3fp = Fp_func(intToBin(self.k3, self.size))
        k4fp = Fp_func(intToBin(self.k4, self.size))
        
        sub_new_k1 = (id_str1 + k4fp + k3fp)
        sub_new_k2 = (k1fp + k4fp + id_str2)
        sub_new_k3 = (id_str1 + k4fp + k2fp)
        sub_new_k4 = (k3fp + k1fp + id_str2)
        
        new_k1int = int(sub_new_k1,2)
        new_k2int = int(sub_new_k2,2)
        new_k3int = int(sub_new_k3,2)
        new_k4int = int(sub_new_k4,2)
        
        self.IDP = (self.IDP ^ self.n2 ^ self.k1)%self.mod
        self.k1 = (self.k1 ^ self.n2 ^ new_k1int)%self.mod
        self.k2 = (self.k2 ^ self.n2 ^ new_k2int)%self.mod
        self.k3 = (self.k3 ^ self.n1 ^ new_k3int)%self.mod
        self.k4 = (self.k4 ^ self.n1 ^ new_k4int)%self.mod
        
        self.n1 = int(rand_bitstream(oracle.size),2)
        self.n2 = int(rand_bitstream(oracle.size),2)
        
        return self.IDP

In [6]:
oracle = EMAPoracle2(96)

def hello(oracle):
    return oracle.helloInternal()   
  
#take in oracle, return a, b, c and oracle (idp)     
def layer2(oracle):
        
    a_int = (oracle.IDP ^ oracle.k1 ^ oracle.n1)#%oracle.mod
    b_int = ((oracle.IDP | oracle.k2) ^ oracle.n1)#%oracle.mod
    c_int = (oracle.IDP ^ oracle.k3 ^ oracle.n2)#%oracle.mod
    
    a = intToBin(a_int, oracle.size)
    b = intToBin(b_int, oracle.size)
    c = intToBin(c_int, oracle.size)
    assert(a_int^oracle.k1^oracle.IDP == oracle.n1)
    
    return a, b, c
        
def layer3(a, b, c, oracle):
    #calc n1, n2
    n1 = a ^ oracle.k1 ^ oracle.IDP 
    n2 = c ^ oracle.IDP ^ oracle.k3
    
    ki = oracle.k1 ^ oracle.k2 ^ oracle.k3 ^ oracle.k4
        
    d_int = ((oracle.IDP & oracle.k4) ^ n2)#%oracle.mod 
    e_int = ((oracle.IDP & n1 | n2) ^ oracle.ID ^ ki)#%oracle.mod
    
    
    d = intToBin(d_int, oracle.size)
    e = intToBin(e_int, oracle.size)

    return d, e

print(layer2(oracle))

('111110010010111010111110011011110000100011010000011111011000001101111110010101111000001100101100', '110110100100010110111000000010101101110111001111011111001110110100100101100000110111101000011001', '001100011011011001101101010111011010010101001100110011011011111010110101010100000101001110010010')


In [7]:
def stage1and2(oracle, IDP):
    
    #Initializing n1, n2, A-E, K1-K4 to all 9s
    n1 = np.array([9]*oracle.size)
    n2 = np.array([9]*oracle.size)
    A = np.array([9]*oracle.size)
    B = np.array([9]*oracle.size)
    C = np.array([9]*oracle.size)
    D = np.array([9]*oracle.size)
    E = np.array([9]*oracle.size)
    K1 = np.array([9]*oracle.size)
    K2 = np.array([9]*oracle.size)
    K3 = np.array([9]*oracle.size)
    K4 = np.array([9]*oracle.size)

    phi = (IDP == 0)
    
    
    tau = (IDP == 1)
    
    
    
    A_str,B_str,C_str = layer2(oracle)
    
#     print("A string: ",A_str)

    D_str,E_str = layer3(int(A_str,2),int(B_str,2),int(C_str,2), oracle)
    
    
    for i in range(oracle.size):
        A[i] = int(A_str[i])
        B[i] = int(B_str[i])
        C[i] = int(C_str[i])
        D[i] = int(D_str[i])
        E[i] = int(E_str[i])

    n2[phi] = D[phi]

    n1[tau] = 1-B[tau]
    
    K1[tau] = A[tau]^n1[tau]^IDP[tau]
    
    A_prime = copy.deepcopy(A)
    A_prime[tau] = A_prime[tau]^IDP[tau]

    B_prime = copy.deepcopy(B)
    B_prime[tau] = B_prime[tau]^IDP[tau]
    
    n1_prime = copy.deepcopy(n1)
    n1_prime[tau] = n1[tau]^IDP[tau]
    
    A_prime_str = ""
    B_prime_str = ""
    
    for i in range(oracle.size):
        A_prime_str += str(A_prime[i])
        B_prime_str += str(B_prime[i])
    
    D_str,E_prime_str = layer3(int(A_prime_str,2),int(B_prime_str,2),int(C_str,2),oracle)
    
    E_prime = np.array([9]*oracle.size)
    
    for i in range(oracle.size):
        E_prime[i] = int(E_prime_str[i])
    
    n2[tau] = E[tau] == E_prime[tau]
    
    K3 = C^IDP^n2
    
    K4[tau] = D[tau]^n2[tau]
    
    K1[tau] = A[tau]^n1[tau]^IDP[tau]
    
    
    return n1,n2,A,B,C,D,E,K1,K2,K3,K4
testOracle = EMAPoracle2(96)
IDP = intToBin(hello(testOracle), testOracle.size)
IDP_np = intToNp(testOracle.IDP, 96)
garbage = stage1and2(testOracle, IDP_np)
print(garbage)

(array([9, 0, 1, 0, 0, 9, 1, 9, 9, 9, 1, 9, 0, 0, 9, 0, 1, 0, 1, 9, 9, 9,
       9, 0, 9, 9, 1, 9, 0, 0, 9, 1, 9, 0, 9, 9, 1, 1, 9, 9, 9, 9, 9, 1,
       1, 0, 9, 0, 9, 9, 9, 9, 1, 1, 9, 9, 1, 0, 0, 1, 0, 1, 0, 9, 1, 1,
       9, 1, 1, 0, 9, 0, 0, 1, 9, 1, 1, 0, 9, 9, 9, 9, 0, 1, 9, 9, 9, 9,
       9, 9, 1, 0, 0, 9, 9, 0]), array([0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1,
       0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1,
       1, 1, 0, 1, 1, 0, 0, 1]), array([1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1,
       0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1,
       1, 1, 0, 1, 1, 0, 1, 1]), array([1, 1, 0, 1, 1, 1,

In [18]:
def EMAP_Cracker(oracle):
    ID = np.array([9]*oracle.size)
    IDP_str = intToBin(hello(oracle), oracle.size)
    IDP = binToNp(IDP_str)
    n1,n2,A,B,C,D,E,K1,K2,K3,K4 = stage1and2(oracle, IDP)
    IDP_new_str = intToBin(hello(oracle), oracle.size)
    IDP_new = binToNp(IDP_new_str)
    numRuns = 0
    while((ID == 9).any()):
        numRuns+=1
        n1_new,n2_new,A_new,B_new,C_new,D_new,E_new,K1_new,K2_new,K3_new,K4_new = stage1and2(oracle, IDP_new)

        K1 = IDP_new ^ IDP ^ n2

        n1 = A ^ IDP ^ K1

        phi = (IDP == 0)

        K2[phi] = B[phi] ^ n1[phi]

        IDP_newest_str = intToBin(hello(oracle), oracle.size)
        IDP_newest = binToNp(IDP_newest_str)

        K1_new = IDP_newest ^ IDP_new ^ n2_new
        n1_new = A_new ^ IDP_new ^ K1_new

        ID[:oracle.size//2] = K1_new[:oracle.size//2] ^ K1[:oracle.size//2] ^ n2[:oracle.size//2]

        K2_new[IDP_new == 0] = B_new[IDP_new == 0] ^ n1_new[IDP_new == 0]
        phi_prime = ((IDP_new[oracle.size//2:] == 0) & (IDP[oracle.size//2:] == 0))
        tau_prime = ((IDP_new[oracle.size//2:] == 1) & (IDP[oracle.size//2:] == 1))
        ID[oracle.size//2:][phi_prime] = K2_new[oracle.size//2:][phi_prime] ^ K2[oracle.size//2:][phi_prime] ^ n2_new[oracle.size//2:][phi_prime]
        ID[oracle.size//2:][tau_prime] = K4_new[oracle.size//2:][tau_prime] ^ K4[oracle.size//2:][tau_prime] ^ n1_new[oracle.size//2:][tau_prime]
        
        IDP = IDP_new
        IDP_new = IDP_newest
        n1,n2,A,B,C,D,E,K1,K2,K3,K4 = n1_new,n2_new,A_new,B_new,C_new,D_new,E_new,K1_new,K2_new,K3_new,K4_new
    return ID, numRuns

testOracle = EMAPoracle2(96)
print(EMAP_Cracker(testOracle))
print(intToBin(oracle.ID, oracle.size))

(array([1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1,
       0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0,
       1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0,
       1, 0, 0, 1, 1, 0, 0, 0]), 9)
101110011011001011100100001101010011110010001001000101001101001001110010110100100001011001110010
